# Computation 4-cell case

Code to compute the transmitted information $I(c; \{ E_A^*, E_B^*, E_C^*, E_D^* \})$ for all the possible combinations of $S_1^A, S_1^B, S_1^C, S_1^D$. These data are necessary to reproduce Fig. 4(b) and 4(c). 

The code takes a long time to run ($\simeq$ 2 days with ncore = 40). To reproduce quickly Fig. 4(b) and 4(c) use Fig_4cells.ipynb with Info_4cells.txt, S1B_4c.txt, S1C_4c.txt, S1D_4c.txt uploaded in https://github.com/rossanabettoni/Information-transmission-NI. 


# Import packages 

In [ ]:
import numpy as np
from scipy.special import xlogy
import multiprocessing as mp 
import os 

In [ ]:
# Set the number of cores for multiprocessing = to the number of available cores
ncore = os.cpu_count()

# Model 

Equations describing the temporal evolution of $R_b$, $Q_b$ and $E^*$ (Eq.1-3 of the main text): 

$\dot{R}_b = k_{d\scriptscriptstyle+}c (R-R_b) - k_{d\scriptscriptstyle-} R_b+ \xi_{R} ;$ 

    
$\dot{Q}_b= k_{e\scriptscriptstyle+} e (Q-Q_b) - k_{e\scriptscriptstyle-} Q_b+ \xi_{Q} ; $ 


$ \dot{E}^* = V_s \frac{R_b^2}{R_b^2 + K_s^2}(E_T- E^*) - \left ( V_{rg}\frac{Q_b}{Q_b + K_{rg}} + k \right ) E^* + \xi_{E}$ 


Standard values of parameters

In [ ]:
# Normalized concentration of molecules c_v = [FGF]/[FGF]_0, e_v = [eph]/[eph]_0
c_v =  5  
e_v =  5 

# Total number of molecules in each cell
Rt_v =  2000 # Total number of FGF receptors
Qt_v =  2000 # Total number of ephrin receptors
Erkt_v= 4000 # Total number of ERK molecules

# Values of the reaction rates  
Kd =  60 # Normalised binding constant of FGF
kd_plus_v = 0.1  
kd_minus_v = Kd*kd_plus_v
 
Ke =  40 # Normalised binding constant of ephrin
ke_plus_v = 0.1 
ke_minus_v = Ke*ke_plus_v

k_v =  0.2 # ERK* de-activation constant
Vs_v=  1*k_v # Maximal rate of ERK activation (by Rb)
Vrg_v = 0.4*k_v # Maximal rate of ERK* deactivation (by Qb)

Ks_v =  200  # Half saturation constant for Rb
Krg_v =  200 # Half saturation constant for Qb

# Cell surface
Scell_v = 6000 # Total cell surface (um^2)
S1max = Scell_v/2 # Maximum surface exposed to FGF
S1_v = 0.5*Scell_v # Surface exposed to FGF (um^2)

# The surface exposed to FGF (S1) is related to the surface exposed to ephrin (S2) through Eq.5:
# S2 = A*S1+B*Scell  
A = -1.1265
B = 0.9092

Function to compute the number of active ERK molecules ($E^*$) (Eq. 1-3)

In [ ]:
def ERK(c, args):  
    
    kd_plus, kd_minus, ke_plus, ke_minus, S1, e, k, Ks, Krg, Vs, Vrg, Qt, Rt, Erkt, Scell = args

    S2= A*S1 +B*Scell
    
    # Cempute Rb_ss, Qb_ss and ERK_ss 
    Rb_ss = Rt*S1/Scell*c/(c+kd_minus/kd_plus)
    Qb_ss = Qt*S2/Scell*e/(e+ke_minus/ke_plus)
    Erk_ss = Erkt* (Vs*Rb_ss**s/(Rb_ss**s+Ks**s)) /(Vs*Rb_ss**s/(Rb_ss**s+Ks**s) + Vrg*Qb_ss/(Qb_ss+Krg)+ k)
    
    del Rb_ss, Qb_ss
    return np.float32(Erk_ss) 

Function to compute the noise in the number of active ERK molecules ($\sigma_e$) (Eq. 22 - 28)

In [ ]:
# Function to compute the noise as a function of the input 
def sigma(c, args):
    
    kd_plus, kd_minus, ke_plus, ke_minus, S1, e, k, Ks, Krg, Vs, Vrg, Qt, Rt, Erkt, Scell = args

    S2= A*S1 +B*Scell
    Rb_ss = Rt*S1/Scell*c/(c+kd_minus/kd_plus)
    Qb_ss = Qt*S2/Scell*e/(e+ke_minus/ke_plus)
    Erk_ss = Erkt* (Vs*Rb_ss**2/(Rb_ss**2+Ks**2)) /(Vs*Rb_ss**2/(Rb_ss**2+Ks**2) + Vrg*Qb_ss/(Qb_ss+Krg)+ k)

    Tau_c = (kd_plus*c+kd_minus)**(-1)
    Tau_e = (ke_plus*e+ke_minus)**(-1)
    Tau_E = (k + Vs*Rb_ss**2/(Rb_ss**2+Ks**2) + Vrg*Qb_ss/(Qb_ss+Krg) )**(-1)

    Gamma_R = 2*Vs*Rb_ss*Ks**2/(Rb_ss**2+Ks**2)**2*(Erkt-Erk_ss)
    Gamma_Q = Vrg*Krg/(Qb_ss+Krg)**2*Erk_ss

    n_ss = c*kd_plus/(kd_minus + c*kd_plus)
    m_ss = e*ke_plus/(ke_minus + e*ke_plus)

    A_R = 2/Tau_c*Rt*S1/Scell*n_ss*(1-n_ss)
    A_Q = 2/Tau_e*Qt*S2/Scell*m_ss*(1-m_ss)
    A_E = 2*Vs*Rb_ss**2/(Rb_ss**2+Ks**2)*(Erkt-Erk_ss)

    sigma_R = Gamma_R**2*A_R/2*(Tau_c-Tau_E)/(Tau_E**(-2)-Tau_c**(-2))
    sigma_Q = Gamma_Q**2*A_Q/2*(Tau_e-Tau_E)/(Tau_E**(-2)-Tau_e**(-2))
    sigma_E = A_E/2*Tau_E

    sigma2 = sigma_R + sigma_Q + sigma_E
        
    return np.float32(np.sqrt(sigma2))

# Information transmission

Input probability distribution $P(c)$: lognormal distribution centered around $\mu_c$, with variance $\sigma_F^2$.

$P(c) = \frac{1}{c \sqrt{2\pi \sigma_F^2}} \exp{\left (-\frac{(\ln{(c)}-\mu_c)^2}{\sigma_F^2} \right )}$   (Eq. 7 of the main text)

In [ ]:
# Lognormal distribution
def Pf_Lognorm(var, mu_c, sigma_F): 
    mu = np.log(mu_c**2/np.sqrt(mu_c**2+sigma_F**2))
    sig = np.sqrt(np.log(1+sigma_F**2/mu_c**2))
    distr = 1/(2*np.pi*sig**2*var**2)**(1/2)*np.exp(-1/2*((np.log(var)-mu)/sig)**2) 
    return np.float32(distr)

Function to compute the conditional probability distribution $P(E^*|c) = \mathcal{G}(E^*, \bar{E}^*, \sigma_e^2)$ (Eq. 9)

In [ ]:
def P_ERKF(var, ERK_temp, args):
    
    # Gaussian centered around ERK_v, with std = sigma_v 
    ERK_v = ERK(var, args= args)
    sigma_v = sigma(var, args=args)

    distr = 1/(2*np.pi*sigma_v**2)**(1/2)*np.exp(-1/2*((ERK_temp - ERK_v)/sigma_v)**2)   
    
    # Return the normalized distribution    
    return distr/np.trapz(distr,ERK_temp)

Function to compute the entropy of $P(\{E_A^*, E_B^*, E_C^*, E_D^*\})$:

$ S[P(\{E_A^*, E_B^*, E_C^*, E_D^*\})] = - \int P(\{E_A^*, E_B^*, E_C^*, E_D^*\}) \log_2(P(\{E_A^*, E_B^*, E_C^*, E_D^*\})) dE_A^* dE_B^* dE_C^* dE_D^*$ 

In [ ]:
def SPERK_4c(ERK_A, ERK_B, ERK_C, ERK_D, c_temp, Pc, argsA, argsB, argsC, argsD):
    
    distr = np.zeros((len(ERK_A), len(ERK_B), len(ERK_C), len(ERK_D)))
    
    for i in range(1, len(c_temp)): 
        PerkfA_1 = np.float32(P_ERKF(c_temp[i], ERK_A, args=argsA))
        PerkfB_1 = np.float32(P_ERKF(c_temp[i], ERK_B, args=argsB))
        PerkfC_1 = np.float32(P_ERKF(c_temp[i], ERK_C, args=argsC))
        PerkfD_1 = np.float32(P_ERKF(c_temp[i], ERK_D, args=argsD))
        
        PerkfA_0 = np.float32(P_ERKF(c_temp[i-1], ERK_A, args=argsA))
        PerkfB_0 = np.float32(P_ERKF(c_temp[i-1], ERK_B, args=argsB))
        PerkfC_0 = np.float32(P_ERKF(c_temp[i-1], ERK_C, args=argsC))
        PerkfD_0 = np.float32(P_ERKF(c_temp[i-1], ERK_D, args=argsD))
            
        # Reshape the arrays to create a 3D matrix with entries P(EA\c)*P(EB\c)*P(EC\c)*P(c)
        PerkfA_1, PerkfB_1, PerkfC_1, PerkfD_1 = np.meshgrid(PerkfA_1, PerkfB_1, PerkfC_1, PerkfD_1, indexing='ij')
        PerkfA_0, PerkfB_0, PerkfC_0, PerkfD_0 = np.meshgrid(PerkfA_0, PerkfB_0, PerkfC_0, PerkfD_0, indexing='ij')
        
        integrand_1 = PerkfA_1*PerkfB_1*PerkfC_1*PerkfD_1*np.float32(Pc[i])
        integrand_0 = PerkfA_0*PerkfB_0*PerkfC_0*PerkfD_0*np.float32(Pc[i-1])
        
        del PerkfA_0, PerkfB_0, PerkfC_0, PerkfD_0, PerkfA_1, PerkfB_1, PerkfC_1, PerkfD_1
        
        delta_c = np.float32(c_temp[i]-c_temp[i-1])
        integral_step = (integrand_1+integrand_0)*delta_c/2
        
        del  integrand_1, integrand_0
        
        distr= distr + integral_step
	
    del integral_step
    
    SPout =  - np.trapz(np.trapz(np.trapz(np.trapz(xlogy(distr, distr)/np.log(2), ERK_A, axis=0), ERK_B, axis=0), ERK_C, axis=0), ERK_D)
    
    del distr
    
    # Return the output matrix
    return SPout

Function to compute information:

$I(c;\{E_A^*, E_B^*, E_C^*, E_D^*\}) =- \int P(\{E_A^*, E_B^*, E_C^*, E_D^*\})\log_2(P(\{E_A^*, E_B^*, E_C^*, E_D^*\})) dE_A^* dE_B^* dE_C^* dE_D^* + $

$ \hspace{5.9 cm} + \int dc P(c) \int P(\{E_A^*, E_B^*, E_C^*, E_D^*\}|c)\log_2(P(\{E_A^*, E_B^*, E_C^*, E_D^*\}|c)) dE_A^* dE_B^* dE_C^* dE_D^* \quad $ (Eq. 30 with N=4)


$ \hspace{4.5 cm} = S[P(\{E_A^*, E_B^*, E_C^*, E_D^*\})] + \int dc P(c) \Big [ \int P(E_A^*|c)\log_2(P(E_A^*|c)) dE_A^*  + \int P(E_B^*|c)\log_2(P(E_B^*|c)) dE_B^* + $

$ \hspace{5.9 cm} + \int P(E_C^*|c)\log_2(P(E_C^*|c)) dE_C^*  +  \int P(E_D^*|c)\log_2(P(E_D^*|c)) dE_D^* \Big ]$ 

$ \hspace{4.5 cm} = S[P(\{E_A^*, E_B^*, E_C^*, E_D^*\})] - \int dc P(c) \Big [ \log_2\left (\sqrt{2\pi e}\sigma_A(c) \right) +  \log_2 \left (\sqrt{2\pi e}\sigma_B(c) \right) + \log_2 \left (\sqrt{2\pi e}\sigma_C(c) \right)  + \log_2 \left (\sqrt{2\pi e}\sigma_D(c) \right) \Big ]$ 

In [ ]:
def I_4cells(c_temp, Pin, SPout, argsA, argsB, argsC, argsD):
     
    sigma_A = np.float32(sigma(c_temp, args = argsA))
    sigma_B = np.float32(sigma(c_temp, args = argsB))
    sigma_C = np.float32(sigma(c_temp, args = argsC))
    sigma_D = np.float32(sigma(c_temp, args = argsD))

    I2= - np.log2((2*np.pi*np.exp(1))**(4/2)*sigma_A*sigma_B*sigma_C*sigma_D)

    # Compute int3 = int P(c)*I2 dc
    I3 = np.trapz(I2*Pin, c_temp)
    
    del sigma_A, sigma_B, sigma_C, sigma_D 
    
    # Return information
    return SPout + I3

Re-write the functions for multiprocessing:

In [ ]:
def SPERK_4c_mp(arg_pout):
    
    ERK_A, ERK_B, ERK_C, ERK_D, c_temp, Pc, argsA, argsB, argsC, argsD = arg_pout
    SPout = SPERK_4c(ERK_A, ERK_B, ERK_C, ERK_D, c_temp, Pc, argsA, argsB, argsC, argsD)
    
    return SPout
    
    
def I_4c_mp(arg_I):
    
    c_temp, Pin, covPout, argsA, argsB, argsC, argsD = arg_I
    Info = I_4cells(c_temp, Pin, SPout, argsA, argsB, argsC, argsD)
    
    return Info

# Computation 

The presence of the constraint imposes:  $S_1^A + S_1^B+ S_1^C + S_1^D = S_1^{tot}$. 

Compute the transmitted information $I(c; \{ E_A^*, E_B^*, E_C^*, E_D^* \})$ for all the possible combinations of $S_1^A, S_1^B, S_1^C, S_1^D$. 


In [ ]:
# The input distribution
mu_c =5
sigma_F= 1

cmin = np.max((mu_c-5*sigma_F, 1e-2))
cmax = mu_c+5*sigma_F
c_temp = np.linspace(cmin, cmax, 50)
P_in =  Pf_Lognorm(c_temp, sigma_F=sigma_F, mu_c=mu_c)

In [ ]:
# Total cell surfaces
ScellA_v = 6000 
ScellB_v = 6000 
ScellC_v = 6000 
ScellD_v = 6000

# Max and min S1 surfaces 
SmaxA = ScellA_v/2
SmaxB = ScellB_v/2
SmaxC = ScellC_v/2
SmaxD = ScellD_v/2

SminA = SmaxA/10
SminB = SmaxB/10
SminC = SmaxC/10
SminD = SmaxD/10

# Max and min S1tot 
S1totmax = SmaxA + SmaxB + SmaxC + SmaxD
S1totmin = SminA + SminB + SminC + SminD
S1tot_temp = np.round(np.linspace(S1totmin, S1totmax, 30), 2)

# Parameters for ERK and sigma functions 
argA = [kd_plus_v, kd_minus_v, ke_plus_v, ke_minus_v, S1_v, e_v, k_v, Ks_v, Krg_v, Vs_v, Vrg_v, Qt_v,\
        Rt_v, Erkt_v, ScellA_v]
argB = [kd_plus_v, kd_minus_v, ke_plus_v, ke_minus_v, S1_v, e_v, k_v, Ks_v, Krg_v, Vs_v, Vrg_v, Qt_v,\
        Rt_v, Erkt_v, ScellB_v]   
argC = [kd_plus_v, kd_minus_v, ke_plus_v, ke_minus_v, S1_v, e_v, k_v, Ks_v, Krg_v, Vs_v, Vrg_v, Qt_v,\
        Rt_v, Erkt_v, ScellC_v]
argD = [kd_plus_v, kd_minus_v, ke_plus_v, ke_minus_v, S1_v, e_v, k_v, Ks_v, Krg_v, Vs_v, Vrg_v, Qt_v,\
        Rt_v, Erkt_v, ScellD_v]

In [ ]:
# Vectors to store the results
Info_S1AS1BS1CS1D_mat = []

# Vectors to store the values of S1A, S1B S1C and S1D for the different values of S1tot
S1A_temp_S1t = []
S1B_temp_S1t = []
S1C_temp_S1t = []
S1D_temp_S1t = []


for k in range(len(S1tot_temp)):
    print('S1tot=', S1tot_temp[k])

    S1A_temp = np.round(np.linspace(SminA, SmaxA, 50),2)
    S1B_temp = np.round(np.linspace(SminB, SmaxB, 50),2)
    S1C_temp = np.round(np.linspace(SminC, SmaxC, 50),2)

    # Create the vectors S1A, S1B, S1C and S1D such that S1A + S1B + S1C + S1D = S1tot
    S1A_3d, S1B_3d, S1C_3d = np.meshgrid(S1A_temp, S1B_temp, S1C_temp, indexing='ij')
    S1D_3d = np.round(S1tot_temp[k]- S1B_3d - S1A_3d- S1C_3d, 2)
    
    S1A_3d[np.where(S1D_3d< SminD)]= 'NaN'
    S1A_3d[np.where(S1D_3d> SmaxD)]= 'NaN'
    S1B_3d[np.where(S1D_3d< SminD)]= 'NaN'
    S1B_3d[np.where(S1D_3d> SmaxD)]= 'NaN'
    S1C_3d[np.where(S1D_3d< SminD)]= 'NaN'
    S1C_3d[np.where(S1D_3d> SmaxD)]= 'NaN'
    S1D_3d[np.where(S1D_3d< SminD)]= 'NaN'
    S1D_3d[np.where(S1D_3d> SmaxD)]= 'NaN'
    
    S1A_temp_S1t.append(S1A_3d)
    S1B_temp_S1t.append(S1B_3d)
    S1C_temp_S1t.append(S1C_3d)
    S1D_temp_S1t.append(S1D_3d)

    S1A_temp = np.ravel(S1A_3d[~np.isnan(S1A_3d)])
    S1B_temp = np.ravel(S1B_3d[~np.isnan(S1B_3d)])
    S1C_temp = np.ravel(S1C_3d[~np.isnan(S1C_3d)])
    S1D_temp = np.ravel(S1D_3d[~np.isnan(S1D_3d)])
    
    del S1B_3d, S1C_3d, S1D_3d
       
    # Update the parameters with the values of S1A_temp, S1B_temp, S1C_temp
    args_tempA = [argA[0:4]+[S1A_temp[i]]+argA[5:] for i in range(len(S1A_temp))]
    args_tempB = [argB[0:4]+[S1B_temp[i]]+argB[5:] for i in range(len(S1B_temp))]
    args_tempC = [argC[0:4]+[S1C_temp[i]]+argC[5:] for i in range(len(S1C_temp))]
    args_tempD = [argD[0:4]+[S1D_temp[i]]+argD[5:] for i in range(len(S1D_temp))]

    # Compute ERK(S1) 
    ERK_S1A = []
    ERK_S1B = []
    ERK_S1C = []
    ERK_S1D = []
    
    for i in range(len(S1A_temp)): 
        
        ERKvminA = ERK(cmin, args_tempA[i])
        ERKvminB = ERK(cmin, args_tempB[i])
        ERKvminC = ERK(cmin, args_tempC[i])
        ERKvminD = ERK(cmin, args_tempD[i])
        
        ERKmax_cellA = ERK(cmax, args_tempA[i])
        ERKmax_cellB = ERK(cmax, args_tempB[i])
        ERKmax_cellC = ERK(cmax, args_tempC[i])
        ERKmax_cellD = ERK(cmax, args_tempD[i])
                            
        ERKmin_cellA = np.max((ERKvminA - ERKvminA/4, 1e-4))
        ERKmin_cellB = np.max((ERKvminB - ERKvminB/4, 1e-4))
        ERKmin_cellC = np.max((ERKvminC - ERKvminC/4, 1e-4))
        ERKmin_cellD = np.max((ERKvminD - ERKvminD/4, 1e-4))
        
        ERK_S1A.append(np.linspace(ERKmin_cellA, ERKmax_cellA+ ERKmax_cellA/4, 50, dtype= np.float32))
        ERK_S1B.append(np.linspace(ERKmin_cellB, ERKmax_cellB+ ERKmax_cellB/4, 50, dtype= np.float32))
        ERK_S1C.append(np.linspace(ERKmin_cellC, ERKmax_cellC+ ERKmax_cellC/4, 50, dtype= np.float32))
        ERK_S1D.append(np.linspace(ERKmin_cellD, ERKmax_cellD+ ERKmax_cellD/4, 50, dtype= np.float32))
        

    # Compute the entropy of the output distribution 
    arg_SPout = []
    
    for l in range(len(S1A_temp)):
        arg_Pout.append((ERK_S1A[l], ERK_S1B[l], ERK_S1C[l], ERK_S1D[l], c_temp, P_in, args_tempA[l],\
                args_tempB[l], args_tempC[l], args_tempD[l]))

    if __name__ == '__main__': 
        p = mp.Pool(ncore)
        SPout_4c = p.map(SPERK_4c_mp, arg_Pout)
    
    del arg_SPout, ERK_S1A, ERK_S1B, ERK_S1C, ERK_S1D
        
    # Compute information as a function of S1A, S1B, S1C, S1D
    print('Computing I')
    arg_I = []
    
    for i in range(len(S1A_temp)):
        arg_I.append((c_temp, P_in, SPout_4c[i], args_tempA[i], args_tempB[i], args_tempC[i], args_tempD[i]))

    if __name__ == '__main__': 
        p = mp.Pool(ncore)
        
        Info_S1AS1BS1CS1D_S1t = p.map(I_4c_mp, arg_I)
 
    Info_S1AS1BS1CS1D_mat_temp = np.zeros((50,50,50))
    Info_S1AS1BS1CS1D_mat_temp[~np.isnan(S1A_3d)] = Info_S1AS1BS1CS1D_S1t
    Info_S1AS1BS1CS1D_mat_temp[Info_S1AS1BS1CS1D_mat_temp == 0.0]= 'NaN'

    Info_S1AS1BS1CS1D_mat.append(Info_S1AS1BS1CS1D_mat_temp)
    
    del Info_S1AS1BS1CS1D_mat_temp, arg_I
    

In [ ]:
# Reshape the arrays (if needed) and save them 
np.savetxt('S1tot_temp.txt', S1tot_temp)

Info_S1AS1BS1CS1D_save = [Info_S1AS1BS1CS1D_mat[i].ravel() for i in range(len(Info_S1AS1BS1CS1D_mat))]
np.savetxt('Info_4cells.txt', Info_S1AS1BS1CS1D_save)
           
S1A_temp_S1t_save = [S1A_temp_S1t[i].ravel() for i in range(len(S1A_temp_S1t))]
np.savetxt('S1A_4c.txt', S1A_temp_S1t_save)

S1B_temp_S1t_save = [S1B_temp_S1t[i].ravel() for i in range(len(S1B_temp_S1t))]
np.savetxt('S1B_4c.txt', S1B_temp_S1t_save)

S1C_temp_S1t_save = [S1C_temp_S1t[i].ravel() for i in range(len(S1C_temp_S1t))]
np.savetxt('S1C_4c.txt', S1C_temp_S1t_save)

S1D_temp_S1t_save = [S1D_temp_S1t[i].ravel() for i in range(len(S1D_temp_S1t))]
np.savetxt('S1D_4c.txt', S1D_temp_S1t_save)


These data can be used to reproduce Fig. 4(b) and 4(c) with the code: Fig_4cells.ipynb